In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
# pd.set_option('display.max_rows', 8)
!date
!whoami

%load_ext autoreload
%autoreload 2

Thu Jul 18 13:26:28 PDT 2019
ndbs


In [2]:
(10.23-7.66)/(2*1.96)

0.6556122448979592

In [3]:
_/2

0.3278061224489796